<a href="https://colab.research.google.com/github/nano-bot01/Movie-Recommendation-System-for-Hollywoord-movies/blob/main/Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation system 

## Importing the dependencies

In [2]:
import numpy as np
import pandas as pd
import difflib
from scipy.sparse import find

from sklearn.feature_extraction.text import TfidfVectorizer 
# for transforming the textual data to numerical feature vector to find present similarity values 
from sklearn.metrics.pairwise import cosine_similarity

## Dataset Loading and Preprocessing


In [3]:
# loading the dataset

dataset = pd.read_json('.\data.json')

In [4]:
dataset.head()

,genre,related-genre,id,tag,image,source,index,name,maturity,duration,synopsis,year,spoilers
0,Action,"[Hindi-Language Movies, Bollywood Movies, Dram...",81595543,Action Movies,https://occ-0-3663-3662.1.nflxso.net/dnm/api/v...,https://www.netflix.com/in/title/81595543,38,Shehzada,U/A 13+,2h 22m,After realizing he was switched at birth by a ...,2023,\nRating 8: Valmiki and Randeep start their ca...
1,Action,"[Crime Movies, Action & Adventure]",81672761,Action Movies,https://occ-0-3663-3662.1.nflxso.net/dnm/api/v...,https://www.netflix.com/in/title/81672761,39,Thunivu,U/A 16+,2h 23m,A major bank heist takes an unnerving turn whe...,2023,"\nRating 8: Radha, a dreaded gangster, and his..."
2,Action,"[Hindi-Language Movies, Bollywood Movies, Acti...",81633804,Action Movies,https://occ-0-3663-3662.1.nflxso.net/dnm/api/v...,https://www.netflix.com/in/title/81633804,40,Mission Majnu,U/A 16+,2h 9m,"In the 1970s, an undercover Indian spy takes o...",2023,"\nRating 8: In 1971, Amandeep ""Aman"" Singh IPS..."
3,Action,"[Movies Based on Books, Action & Adventure, US...",80230399,Action Movies,https://occ-0-3663-3662.1.nflxso.net/dnm/api/v...,https://www.netflix.com/in/title/80230399,41,Extraction,A,1h 58m,A hardened gun-for-hire's latest mission becom...,2020,\nRating 3: Ovi Mahajan – son of incarcerated ...
4,Action,[Action & Adventure],81508719,Action Movies,https://occ-0-3663-3662.1.nflxso.net/dnm/api/v...,https://www.netflix.com/in/title/81508719,42,Raw,U/A 16+,2h 36m,A jaded former intelligence agent is pulled ba...,2022,\nRating 3: Lifelong vegetarian Justine begins...


In [5]:
dataset.tail()

,genre,related-genre,id,tag,image,source,index,name,maturity,duration,synopsis,year,spoilers
35,Drama,"[Family Features, Dramas, Movies Based on Book...",80993105,Action Movies,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",https://www.netflix.com/in/title/80993105,73,Mowgli: Legend of the Jungle,U/A 13+,1h 44m,An orphaned boy raised by animals in the jungl...,2018,"\nRating 7: In the jungles of India, the gigan..."
36,Action,"[Crime Movies, Comedies, Movies Based on Books...",81511779,Action Movies,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",https://www.netflix.com/in/title/81511779,74,Bullet Train,A,2h 6m,Five assassins board a Japanese bullet train b...,2022,"\nRating 3: Yuichi Kimura ""The Father"" boards ..."
37,Drama,"[Dramas, Movies Based on Real Life, Action & A...",81242575,Action Movies,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",https://www.netflix.com/in/title/81242575,75,Major,U/A 16+,2h 25m,This biopic follows the life of Indian Army of...,2022,\nRating 10: The movie starts with Sandeep Unn...
38,Action,"[Action & Adventure, US Movies]",70079583,Action Movies,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",https://www.netflix.com/in/title/70079583,76,The Dark Knight,U/A 16+,2h 32m,"Batman, Lieutenant Gordon and District Attorne...",2008,\nRating 3: A gang of masked criminals robs a ...
39,Fantasy,"[Family Features, Movies Based on Books, Actio...",70178621,Action Movies,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",https://www.netflix.com/in/title/70178621,77,Journey 2: The Mysterious Island,U/A 13+,1h 34m,When a distress signal pinpoints his grandfath...,2012,\nRating 10: Four years after his adventure in...


In [6]:
# columns in dataset

dataset.columns

Index(['genre', 'related-genre', 'id', 'tag', 'image', 'source', 'index',
       'name', 'maturity', 'duration', 'synopsis', 'year', 'spoilers'],
      dtype='object')

In [7]:
# shape of dataset frame

dataset.shape

(40, 13)

In [8]:
# information about dataset

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   genre          40 non-null     object
 1   related-genre  40 non-null     object
 2   id             40 non-null     int64 
 3   tag            40 non-null     object
 4   image          40 non-null     object
 5   source         40 non-null     object
 6   index          40 non-null     int64 
 7   name           40 non-null     object
 8   maturity       40 non-null     object
 9   duration       40 non-null     object
 10  synopsis       40 non-null     object
 11  year           40 non-null     int64 
 12  spoilers       40 non-null     object
dtypes: int64(3), object(10)
memory usage: 4.2+ KB


In [78]:
# checking for null values in dataset

dataset.isnull().sum()

id               0
tag              0
image            0
source           0
name             0
genre            0
maturity         0
duration         0
synopsis         0
related-genre    0
plot             0
dtype: int64

In [79]:
# Total count of null values

dataset.isnull().sum().sum()

0

In [80]:
# changing the title of movies to lowercase 

dataset['name'] = dataset['name'].str.lower()
print(dataset['name'].head())

0                            time trap
1    spider-man: into the spider-verse
2                                 lucy
3                    godzilla vs. kong
4                johnny english reborn
Name: name, dtype: object


In [81]:
selected_features = ['tag', 'maturity', 'synopsis','duration','related-genre']
print(selected_features)

['tag', 'maturity', 'synopsis', 'duration', 'related-genre']


In [82]:
# replacing the null values with null string

for features in selected_features :
  dataset[features] = dataset[features].fillna('')

### Combining the selected features 

In [83]:
dataset['duration'] = dataset['duartion'].astype(str)

KeyError: 'duartion'

In [84]:
list(dataset['related-genre'])

[['Sci-Fi Movies', 'US Movies', 'Action & Adventure'],
 ['US Movies', 'Action & Adventure', 'Comedies'],
 ['Sci-Fi Movies', 'Action & Adventure'],
 ['Sci-Fi Movies', 'US Movies', 'Action & Adventure'],
 ['Spy Movies', 'Family Features', 'Action & Adventure', 'Comedies'],
 ['Crime Movies', 'Action & Adventure'],
 ['US Movies', 'Action & Adventure'],
 ['Family Features',
  'US Movies',
  'Teen Movies',
  'Movies Based on Books',
  'Action & Adventure',
  'Comedies'],
 ['Family Features',
  'US Movies',
  'Movies Based on Books',
  'Action & Adventure'],
 ['Sci-Fi Movies', 'US Movies', 'Action & Adventure'],
 ['US Movies', 'Action & Adventure'],
 ['Hindi-Language Movies',
  'Bollywood Movies',
  'Crime Movies',
  'Action & Adventure',
  'Dramas'],
 ['Sci-Fi Movies', 'Family Features', 'US Movies', 'Action & Adventure'],
 ['Hindi-Language Movies',
  'Bollywood Movies',
  'Crime Movies',
  'Action & Adventure'],
 ['Romantic Comedies',
  'Hindi-Language Movies',
  'Bollywood Movies',
  'Acti

In [85]:
combined_features = dataset['genre']+''+dataset['maturity']+''+dataset['synopsis']+''+''+dataset['duration']+''

In [86]:
combined_features.head()

0    Sci-Fi MoviesU/A 13+ To find their missing pro...
1    US MoviesU/A 13+ After being bitten by a radio...
2    Sci-Fi MoviesA When a young American in Taiwan...
3    Sci-Fi MoviesU/A 13+ An ancient rivalry pits t...
4    ComediesU/A 13+ Bumbling spy Johnny English sh...
dtype: object

In [87]:
print(combined_features )

0     Sci-Fi MoviesU/A 13+ To find their missing pro...
1     US MoviesU/A 13+ After being bitten by a radio...
2     Sci-Fi MoviesA When a young American in Taiwan...
3     Sci-Fi MoviesU/A 13+ An ancient rivalry pits t...
4     ComediesU/A 13+ Bumbling spy Johnny English sh...
5     Crime MoviesA A former agent with a troubled p...
6     US MoviesA King Leonidas of Sparta leads 300 p...
7     Movies Based on BooksU/A 13+ Four high school ...
8     Movies Based on BooksU/A 13+ When a distress s...
9     Sci-Fi MoviesU/A 16+ When the world's defenses...
10    US MoviesU/A 16+ When an American archaeology ...
11    Bollywood MoviesA After street smart kid Raees...
12    Sci-Fi MoviesU/A 13+ When his loved ones are p...
13    Bollywood MoviesU/A 16+ A fearless, faithful (...
14    Bollywood MoviesU/A 13+ What could have been a...
15    Bollywood MoviesU/A 16+ After a troubled past ...
16    Sci-Fi MoviesU/A 16+ When a theme park's newes...
17    DramasA A fearless warrior on a perilous m

Converting the text data into features vector

In [88]:
vectorizer = TfidfVectorizer()

feature_vector = vectorizer.fit_transform(combined_features )

In [89]:
print(feature_vector)

  (0, 14)	0.20597432975902552
  (0, 6)	0.11427126370652695
  (0, 428)	0.22907704984755142
  (0, 61)	0.22907704984755142
  (0, 519)	0.20597432975902552
  (0, 363)	0.22907704984755142
  (0, 111)	0.22907704984755142
  (0, 546)	0.22907704984755142
  (0, 67)	0.09787962294278575
  (0, 185)	0.22907704984755142
  (0, 265)	0.16647996890675842
  (0, 575)	0.20597432975902552
  (0, 135)	0.22907704984755142
  (0, 520)	0.07870800066315313
  (0, 286)	0.13737398379505286
  (0, 558)	0.22907704984755142
  (0, 500)	0.20597432975902552
  (0, 371)	0.10713967827724467
  (0, 257)	0.20597432975902552
  (0, 410)	0.22907704984755142
  (0, 346)	0.20597432975902552
  (0, 521)	0.3153934447353747
  (0, 233)	0.17686834464731996
  (0, 532)	0.06934628906882165
  (0, 1)	0.10388288796596543
  :	:
  (39, 211)	0.2255396429171208
  (39, 214)	0.2255396429171208
  (39, 184)	0.2255396429171208
  (39, 141)	0.2255396429171208
  (39, 145)	0.2255396429171208
  (39, 192)	0.2255396429171208
  (39, 405)	0.2255396429171208
  (39, 447

### Cosine Similiarity 

In [90]:
# similarity confidence value using cosine similiarity

similarity = cosine_similarity(feature_vector)


In [91]:
print(similarity)

[[1.         0.03125017 0.08033943 ... 0.09279189 0.09225071 0.11322192]
 [0.03125017 1.         0.02602955 ... 0.08094194 0.02987701 0.00747572]
 [0.08033943 0.02602955 1.         ... 0.0539164  0.0128249  0.0699072 ]
 ...
 [0.09279189 0.08094194 0.0539164  ... 1.         0.09497432 0.07669033]
 [0.09225071 0.02987701 0.0128249  ... 0.09497432 1.         0.02768691]
 [0.11322192 0.00747572 0.0699072  ... 0.07669033 0.02768691 1.        ]]


In [92]:
print(similarity.shape)
# compare with all the other movies present there

(40, 40)


In [93]:
# creating a list with all the moviee names in dataset

list_of_all_titles = dataset['name'].tolist()
print(list_of_all_titles)

['time trap', 'spider-man: into the spider-verse', 'lucy', 'godzilla vs. kong', 'johnny english reborn', 'the ghost', '300', 'jumanji: welcome to the jungle', 'journey 2: the mysterious island', 'pacific rim', 'the mummy', 'raees', 'spider-man: no way home', 'sooryavanshi', 'chennai express', 'brothers', 'jurassic world', 'rrr', 'transformers: the last knight', 'jumanji: the next level', 'uncharted', 'rush hour', 'naam shabana', 'mission majnu', 'bullet train', 'major', 'red notice', 'lakshya', 'extraction', 'godfather', 'baahubali 2: the conclusion', 'thallumaala', 'baahubali: the beginning', 'edge of tomorrow', 'troll', 'the dark knight', 'man of steel', 'the amazing spider-man 2', 'don 2', 'jurassic park']


## Recommedation

In [94]:
# getting the name as input for recommendation

movie_name = input('Enter your favourite movie name : ')

In [95]:
# finding the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['don 2']


In [96]:
close_match = find_close_match[0]
print(close_match)

don 2


In [97]:
# Index of movie with title and data

index_of_movie = dataset[dataset.name == close_match]['id'].values[0]
# we are finding the index of movie by comparing the title of each movie with closly matched string and returing its index 
print(index_of_movie)

70224999


In [98]:
# finding similariy score 

score = list(enumerate(similarity[1]))
print(score)

[(0, 0.03125017210483985), (1, 1.0), (2, 0.026029546958858903), (3, 0.0324815007339435), (4, 0.05685322563196542), (5, 0.04296671068344504), (6, 0.07055685187318822), (7, 0.021557843697904738), (8, 0.08508674865603816), (9, 0.007413792750118166), (10, 0.033672426405641505), (11, 0.06068565086446753), (12, 0.03756926228492297), (13, 0.018659894957695794), (14, 0.04092024072161702), (15, 0.04938519533836644), (16, 0.049801003099139436), (17, 0.024061078430805467), (18, 0.04712756904107665), (19, 0.009871042420299786), (20, 0.08980381690339587), (21, 0.038585931525419406), (22, 0.03918298603097857), (23, 0.035508510320878255), (24, 0.0), (25, 0.04190824238280394), (26, 0.07402241266918612), (27, 0.0876396277504515), (28, 0.013227664518598059), (29, 0.011936345971085666), (30, 0.025960138339206865), (31, 0.01050048040549998), (32, 0.037678084750555604), (33, 0.041037295875575794), (34, 0.056857385440446113), (35, 0.045377070097912565), (36, 0.04840396430904845), (37, 0.08094193742369458), 

In [ ]:
len(score)

40

In [104]:
# finding movies having higher similarity score 

sorted_list = sorted(score, key = lambda x:x[1], reverse = True)
# sort based on non increasing order, key is based on first index of score in list element which is score in (index, score)
print(sorted_list)

IndexError: tuple index out of range

In [101]:
print(sorted_list[0], sorted_list[1], sorted_list[2], sorted_list[3], sorted_list[4])
# top five score with index 

(1, 1.0) (20, 0.08980381690339587) (27, 0.0876396277504515) (8, 0.08508674865603816) (37, 0.08094193742369458)


In [102]:
# printing the name of simila rmovies based on the index

print("Movies Results : \n")

i = 1

for movie in sorted_list:
  index = movie[0]
  title_of_movie = dataset[dataset.index == index]['name'].values[0]
  if(i<=5):
    print(i, '.', title_of_movie)
    i += 1

Movies Results : 

1 . spider-man: into the spider-verse
2 . uncharted
3 . lakshya
4 . journey 2: the mysterious island
5 . the amazing spider-man 2


## Recommendation function 

In [ ]:
def recommend(name):
  find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
  
  try:
    close_match = find_close_match[0]
  except IndexError:
    close_match = 'null'
    print("No result found with this title")
    return 
  index_of_movie = dataset[dataset.name == close_match]['name'].values[0]
  score = list(enumerate(similarity[1]))
  sorted_list = sorted(score, key = lambda x:x[1], reverse = True)


  print("Movies Results : \n")

  i = 1

  for movie in sorted_list:
    index = movie[0]
    title_of_movie = dataset[dataset.index == index]['name'].values[0]
    if(i<=5):
      print(i, '. ', title_of_movie)
      i += 1

In [ ]:
movie_name = input('Enter your favourite movie name : ')

recommend(movie_name)

Movies Results : 

1 .  spider-man: into the spider-verse
2 .  uncharted
3 .  lakshya
4 .  journey 2: the mysterious island
5 .  the amazing spider-man 2
